In [208]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

#kaggle.com/code/kritanjalijain/apriori-algorithm-hotel-management-system

In [209]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [210]:
# load data:
df_ht = pd.read_csv("hotel_bookings.csv")

df_ht.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [211]:
# Replace missing values:
# agent: If no agency is given, booking was most likely made without one.
# company: If none given, it was most likely private.
# rest schould be self-explanatory.


nan_replacements = {"children:": 0.0,"country": "Unknown", "agent": 0, "company": 0}
df_clean = df_ht.fillna(nan_replacements)

# "meal" contains values "Undefined", which is equal to SC.
df_clean["meal"].replace("Undefined", "SC", inplace=True)

# Some rows contain entreis with 0 adults, 0 children and 0 babies. 
# I'm dropping these entries with no guests.
zero_guests = list(df_clean.loc[df_clean["adults"]
                   + df_clean["children"]
                   + df_clean["babies"]==0].index)

df_clean.drop(df_clean.index[zero_guests], inplace=True)


#
df_clean = df_clean[df_clean['is_canceled'] == 0]

df_clean.head(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,0.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,0.0,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [212]:
# df_clean

In [213]:
# portugal_df = df_clean[df_clean["country"] == 'PRT']
# france_df = df_clean[df_clean["country"] == 'FRA']
# england_df = df_clean[df_clean["country"] == 'GBR']
# germany_df = df_clean[df_clean["country"] == 'DEU']
# italy_df = df_clean[df_clean["country"] == 'ITA']

df_clean = pd.get_dummies(data=df_clean,
               columns=['hotel','meal','market_segment','distribution_channel','country'],
               drop_first=False,
               dtype=None
               )
df_clean.columns.values

array(['is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date',
       'hotel_City Hotel', 'hotel_Resort Hotel', 'meal_BB', 'meal_FB',
       'meal_HB', 'meal_SC', 'market_segment_Aviation',
       'market_segment_Complementary', 'market_segment_Corporate',
       'market_segment_Direct', 'market_segment_Groups',
       'market_segment_Offline TA/TO', 'market_segment_Online TA',
       'distribution_channel_Corporate', 'distribution_channel_D

In [224]:
clean_france_df = df_clean[[  
       'children', 
       'babies', 
       'previous_cancellations',
       'previous_bookings_not_canceled',  
       'booking_changes', 
       'required_car_parking_spaces', 
       'total_of_special_requests',
       'hotel_City Hotel', 'hotel_Resort Hotel',
       'country_FRA','country_PRT','country_GBR','country_ESP','country_DEU',
       'meal_BB', 'meal_FB','meal_HB', 'meal_SC',
        'market_segment_Aviation','market_segment_Complementary', 
        # 'market_segment_Corporate',
    #    'market_segment_Direct',
       'market_segment_Groups',
    #    'market_segment_Offline TA/TO', 'market_segment_Online TA',
       'distribution_channel_Corporate', 'distribution_channel_Direct',
       'distribution_channel_GDS'
    #    , 'distribution_channel_TA/TO'
       ]]
#df_clean.dropna(axis=0, subset =)
clean_france_df.dropna()

,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,required_car_parking_spaces,total_of_special_requests,hotel_City Hotel,hotel_Resort Hotel,country_FRA,...,meal_BB,meal_FB,meal_HB,meal_SC,market_segment_Aviation,market_segment_Complementary,market_segment_Groups,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS
0,0.0,0,0,0,3,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,0.0,0,0,0,4,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
2,0.0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
3,0.0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4,0.0,0,0,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0.0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
119386,0.0,0,0,0,0,0,2,1,0,1,...,1,0,0,0,0,0,0,0,0,0
119387,0.0,0,0,0,0,0,4,1,0,0,...,1,0,0,0,0,0,0,0,0,0
119388,0.0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [225]:
# Defining the hot encoding function to make the data suitable
# for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0   
    if(x>= 1):
        return 1

# Encoding the datasets
clean_france_df_encoded = clean_france_df.applymap(hot_encode)
clean_france_df = clean_france_df_encoded.copy()

In [226]:
# Building the model
frq_items = apriori(clean_france_df, min_support = 0.05, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
pd.set_option('display.max_row', 500)
rules.head(500)


#
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequent_len"] = rules["consequents"].apply(lambda x: len(x))


In [241]:
rules_filtered1 = rules[
        (rules['antecedent_len'] == 1) &
        # (rules['consequent_len'] == 1) &
        (rules['support'] > 0.05) &
        (rules['confidence'] > 0.5) &
        (rules['lift'] > 1.15)  
    ]

rules_filtered1.head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
41,(distribution_channel_Corporate),(meal_BB),0.069110,0.769620,0.066444,0.961420,1.249213,0.013255,5.971438,1,1
24,(meal_SC),(hotel_City Hotel),0.099799,0.614363,0.087041,0.872161,1.419618,0.025728,3.016590,1,1
19,(country_FRA),(hotel_City Hotel),0.112890,0.614363,0.094240,0.834790,1.358789,0.024884,2.334219,1,1
20,(country_DEU),(hotel_City Hotel),0.080881,0.614363,0.066790,0.825779,1.344121,0.017100,2.213489,1,1
9,(required_car_parking_spaces),(hotel_Resort Hotel),0.098772,0.385637,0.073176,0.740856,1.921123,0.035086,2.370738,1,1
85,(country_FRA),"(meal_BB, hotel_City Hotel)",0.112890,0.474197,0.076535,0.677964,1.429709,0.023003,1.632746,1,2
29,(country_GBR),(hotel_Resort Hotel),0.128888,0.385637,0.078948,0.612536,1.588376,0.029245,1.585602,1,1
53,(required_car_parking_spaces),"(hotel_Resort Hotel, meal_BB)",0.098772,0.295423,0.060284,0.610339,2.065980,0.031105,1.808178,1,2
16,(meal_SC),(total_of_special_requests),0.099799,0.511178,0.059338,0.594577,1.163149,0.008323,1.205706,1,1
31,(meal_HB),(hotel_Resort Hotel),0.126315,0.385637,0.073309,0.580369,1.504964,0.024598,1.464057,1,1


In [242]:
rules_filtered2 = rules[ 
      (rules['antecedent_len'] == 2) &
    #   (rules['consequent_len'] == 2) &
      (rules['support'] > 0.05) &
       (rules['confidence'] > 0.5) &
       (rules['lift'] > 1.2)  
    ]

rules_filtered2.head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
64,"(meal_SC, total_of_special_requests)",(hotel_City Hotel),0.059338,0.614363,0.057298,0.965626,1.571750,0.020843,11.218753,2,1
81,"(meal_BB, country_FRA)",(hotel_City Hotel),0.088560,0.614363,0.076535,0.864218,1.406689,0.022127,2.840117,2,1
56,"(total_of_special_requests, country_FRA)",(hotel_City Hotel),0.062591,0.614363,0.052899,0.845154,1.375659,0.014445,2.490460,2,1
50,"(meal_BB, required_car_parking_spaces)",(hotel_Resort Hotel),0.081428,0.385637,0.060284,0.740341,1.919787,0.028883,2.366034,2,1
65,"(meal_SC, hotel_City Hotel)",(total_of_special_requests),0.087041,0.511178,0.057298,0.658294,1.287797,0.012805,1.430532,2,1
97,"(meal_BB, country_GBR)",(hotel_Resort Hotel),0.097479,0.385637,0.060644,0.622128,1.613249,0.023053,1.625850,2,1
71,"(country_PRT, total_of_special_requests)",(hotel_Resort Hotel),0.116983,0.385637,0.059871,0.511795,1.327142,0.014758,1.258412,2,1
102,"(meal_BB, distribution_channel_Direct)",(hotel_Resort Hotel),0.134714,0.385637,0.068083,0.505393,1.310542,0.016133,1.242125,2,1


# ADR관련

In [235]:
rules_filtered2 = rules[ 
      (rules['antecedent_len'] == 2) &
      (rules['support'] > 0.05) &
       (rules['confidence'] > 0.5) &
       (rules['lift'] > 1.2)  
    ]

rules_filtered2.head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
64,"(meal_SC, total_of_special_requests)",(hotel_City Hotel),0.059338,0.614363,0.057298,0.965626,1.571750,0.020843,11.218753,2,1
81,"(meal_BB, country_FRA)",(hotel_City Hotel),0.088560,0.614363,0.076535,0.864218,1.406689,0.022127,2.840117,2,1
56,"(total_of_special_requests, country_FRA)",(hotel_City Hotel),0.062591,0.614363,0.052899,0.845154,1.375659,0.014445,2.490460,2,1
50,"(meal_BB, required_car_parking_spaces)",(hotel_Resort Hotel),0.081428,0.385637,0.060284,0.740341,1.919787,0.028883,2.366034,2,1
65,"(meal_SC, hotel_City Hotel)",(total_of_special_requests),0.087041,0.511178,0.057298,0.658294,1.287797,0.012805,1.430532,2,1
97,"(meal_BB, country_GBR)",(hotel_Resort Hotel),0.097479,0.385637,0.060644,0.622128,1.613249,0.023053,1.625850,2,1
71,"(country_PRT, total_of_special_requests)",(hotel_Resort Hotel),0.116983,0.385637,0.059871,0.511795,1.327142,0.014758,1.258412,2,1
102,"(meal_BB, distribution_channel_Direct)",(hotel_Resort Hotel),0.134714,0.385637,0.068083,0.505393,1.310542,0.016133,1.242125,2,1


In [220]:
# for i in range(0,len(dataset)):
#     transactions.append({str(dataset.values[i,j]) for j in range(0,len(dataset.columns)) if str(dataset.values[i,j]) != 'nan'})

# print('Transactions:\n')
# for transaction in transactions:
#     print(transaction)

1. customers travel and stay with children have more special requests
2. customers who travel and stay during weekdays have more special requests

In [221]:
# def init_candidates(transactions):
#     candidates = dict()
#     for transaction in transactions:
#         for item in transaction:
#             itemset = set()
#             itemset.add(item)
#             itemset = frozenset(itemset)
#             if itemset not in candidates:
#                 candidates[itemset] = 1
#             else:
#                 candidates[itemset] += 1
                
#     return candidates


# def prune_candidates(candidates,support):
#     fp = dict()
#     for key in candidates:
#         if candidates[key] >= support:
#             fp[key] = candidates[key]
    
#     return fp


# def get_count(transactions,itemset):
#     itemset = set(itemset)
#     count = 0
#     for transaction in transactions:
#         if itemset.issubset(transaction):
#             count += 1
            
#     return count


# def get_candidates(fp,transactions,fp_length):
#     candidates = dict()
#     for key1 in fp:
#         for key2 in fp:
#             if key2 != key1:
#                 itemset1 = set(key1)
#                 itemset2 = set(key2)
#                 itemset = itemset1.union(itemset2)
#                 itemset = frozenset(itemset)
#                 if itemset not in candidates and len(itemset) == fp_length:
#                     candidates[itemset] = get_count(transactions,itemset)
    
#     return candidates


# def get_itemset_length(candidates):
#     itemset_len = 0
#     for key in candidates:
#         itemset_len = len(key)
#         break
    
#     return itemset_len


# import itertools

# def get_rules(fp):
#     rules = []
#     for itemset in fp:
#         itemset = set(itemset)
#         for length in range(1,len(itemset)):
#             subsets = set(itertools.combinations(itemset, length))
#             for subset in subsets:
#                 lhs = set(subset)
#                 rhs = itemset.difference(lhs)
#                 rules.append([lhs,rhs])
    
#     return rules


# def prune_rules(rules,confidence,transactions):
#     fp = []
#     for rule in rules:
#         lhs = rule[0]
#         rhs = rule[1]
#         lhs_rhs = lhs.union(rhs)
#         lhs_count = get_count(transactions,lhs)
#         lhs_rhs_count = get_count(transactions,lhs_rhs)
#         conf = lhs_rhs_count/lhs_count
#         if conf >= confidence:
#             support = lhs_rhs_count/len(transactions)
#             fp.append([lhs,rhs,support,conf])
    
#     return fp

# def display(candidates,fp,length):
#     print()
#     print("****** {}-Frequent Candidates ******".format(length))
#     for key in candidates:
#         print(set(key),' : ',candidates[key])
#     print()
#     print("****** {}-Frequent Itemlist ******".format(length))
#     for key in fp:
#         print(set(key),' : ',fp[key])
# def apriori(transactions,support,confidence):
#     support = support/100 * len(transactions)
#     confidence = confidence/100
#     candidates = init_candidates(transactions)
#     old_fp = prune_candidates(candidates,support)
#     final_fp = dict()
#     display(candidates,old_fp,1)
    
#     while True:
#         new_fp_length = get_itemset_length(candidates) + 1
#         candidates = get_candidates(old_fp,transactions,new_fp_length)
#         new_fp = prune_candidates(candidates,support)
#         display(candidates,new_fp,new_fp_length)
#         if len(new_fp)<1:
#             final_fp = old_fp
#             break
#         else:
#             old_fp = new_fp
    
#     rules = get_rules(final_fp)
#     fp = prune_rules(rules,confidence,transactions)
#     return fp
# support = float(input('Enter support percentage : '))
# confidence = float(input('Enter confidence percentage : '))
# fp = apriori(transactions,support,confidence)